### Objective: this notebook fits behavioral data to model 1 (baseline model, 1 LR), calculates the BIC, and extracts RPEs

In [2]:
import numpy as np  

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob
import math
import os

from scipy.stats import norm
from scipy.optimize import minimize

##### load recoded data

In [3]:
os.getcwd()

'/Users/jackiebeltran/Documents/GitHub/RP_Learning/1_scripts/model_1'

###### ALL Subjects

In [70]:
flist = glob.glob('../../0_data/N_77/*.csv')
print('participants in this dataset are: ' + str(len(flist)))

participants in this dataset are: 77


###### HEALTHY CONTROLS

In [4]:
flist = glob.glob('../../0_data/healthy/*.csv')
print('HC participants in this dataset are: ' + str(len(flist)))

HC participants in this dataset are: 38


###### MDD

In [20]:
mlist = glob.glob('../../0_data/mdd/*.csv')
print('MDD participants in this dataset are: ' + str(len(mlist)))

MDD participants in this dataset are: 39


##### define functions

In [12]:
def m1_loglikelihood(training_params, actions, rewards, condition):
    
    alpha = training_params[0]
    beta = training_params[1]
    
    q_value = np.ones((3,2))*0.0 
    T = len(actions)
    L = 0  
    
    for t in range(T): 
        
        # compute choice probabilities of picking an action based on soft max 
        p = np.exp(beta * q_value[condition[t]-1,:]) / (np.exp(beta * q_value[condition[t]-1,:])).sum()

        # compute choice probability for actual choice based on the probability computed above by condition 
        choiceProb = p[actions[t]-1]
        #print('choice probability is ' + str(choiceProb))

        # sum of the natural log of each individual choice probability to get the prob of a whole dataset (90 trials)
        L += np.log(choiceProb) 

        # update values with q learning, index for t and update by condition 
        if ((condition[t] == 1) | (condition[t] == 2)):
            q_value[condition[t]-1, actions[t]-1] = q_value[condition[t]-1, actions[t]-1] + alpha * (rewards[t] - q_value[condition[t]-1, actions[t]-1])
        
    return -L   


In [7]:
def fit_RP_Learning(actions, rewards, condition):
    
    init_cond = np.array([np.random.uniform(0,1), # learning rate
                        np.random.randint(1,50)]) # beta
    
    bnds = [(1e-6,1), (1,50)]
        
    optimum_output = minimize(m1_loglikelihood, init_cond, args=(actions, rewards, condition), method='L-BFGS-B',bounds=bnds)
    
    return optimum_output, init_cond

In [8]:
def model1_BIC(actions, rewards, condition):
    
    init_cond = np.array([np.random.uniform(0,1), # learning rate
                          np.random.uniform(1,50)])  # beta
    
    bnds = [(1e-6,1), (1,50)]
    km = len(bnds) 
   
    optimum_output = minimize(m1_loglikelihood, init_cond, args=(actions, rewards, condition), method='L-BFGS-B', bounds=bnds)
    
    neg_loglikelihood = optimum_output.fun
    
    BIC = km * np.log(len(actions)) + 2*neg_loglikelihood
    
    return BIC

In [10]:
def fitting(filename):    
    
    parameters = [] # empty list to store results

    a = os.path.basename(filename)
    subID = os.path.splitext(a)[0]

    # load data & subset for columns of interest
    df = pd.read_csv(filename)

    col_name = ['sub_ID', 'actions','rewards', 'condition']   
    sub_df = df[col_name]

    # Obtain actions, rewards, conditions
    condition = sub_df["condition"]
    actions = sub_df['actions']
    rewards = sub_df['rewards']

    # compute best fit parameters
    res, init_cond = fit_RP_Learning(actions, rewards, condition)
    
    initial_values = init_cond
    fit_params = np.array([res.x[0], res.x[1]])

    max_LL = m1_loglikelihood(fit_params, actions, rewards, condition) 

    LL_per_trial = np.exp(-max_LL/len(actions))
    
    ## Model Recovery 
    BIC_model1 = model1_BIC(actions, rewards, condition) 

    results_df = pd.DataFrame({
    'sub_ID': subID,
    'initial_alpha_conditions' : initial_values[0],
    'initial_beta_conditions' : initial_values[1],
    'fit_alpha': res.x[0],
    'fit_beta' : res.x[1],
    'log_likelihood': -max_LL,
    'BIC_model1': BIC_model1,
    'likelihood_per_trial': LL_per_trial}, 
    index=pd.Index(sub_df['sub_ID'].unique(), name='participant_id'))

    parameters.append(results_df)


    final_result_df = pd.concat(parameters, ignore_index=True)
    
    return final_result_df

#### fit the data 

In [71]:
n_iter = 5
final_df = pd.DataFrame()

for j in np.arange(n_iter) + 1:
    
    ### flist is for HEALTHY CONTROLS 
    for i in flist:

    ### mlist for MDD 
    #for i in mlist:

        iteration_results = fitting(i)
        
        # add iteration column 
        iteration_results['Iteration'] = j
                
        final_df = final_df.append(iteration_results, ignore_index=True)
        final_df = final_df.sort_values(by=["sub_ID", "Iteration"])

final_df

/var/folders/jc/p5x3b5x57nj1x3fy75x85xwr0000gn/T/ipykernel_52304/1731646800.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(iteration_results, ignore_index=True)
/var/folders/jc/p5x3b5x57nj1x3fy75x85xwr0000gn/T/ipykernel_52304/1731646800.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(iteration_results, ignore_index=True)
/var/folders/jc/p5x3b5x57nj1x3fy75x85xwr0000gn/T/ipykernel_52304/1731646800.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(iteration_results, ignore_index=True)
/var/folders/jc/p5x3b5x57nj1x3fy75x85xwr0000gn/T/ipykernel_52304/1731646800.py:17: FutureWarning: The frame.append method is deprecated and will be rem

,sub_ID,initial_alpha_conditions,initial_beta_conditions,fit_alpha,fit_beta,log_likelihood,BIC_model1,likelihood_per_trial,Iteration
0,401,0.009429,17.0,0.078436,17.025370,-37.830120,84.390029,0.656827,1
1,401,0.485505,39.0,0.222628,5.801814,-37.695205,84.390029,0.657812,2
2,401,0.499582,18.0,0.222629,5.801799,-37.695205,84.390029,0.657812,3
3,401,0.010662,4.0,0.222629,5.801810,-37.695205,84.390029,0.657812,4
4,401,0.734983,15.0,0.222628,5.801811,-37.695205,84.390029,0.657812,5
...,...,...,...,...,...,...,...,...,...
379,549,0.102355,30.0,0.281737,28.342976,-27.720559,64.440737,0.734911,1
380,549,0.723549,12.0,0.281726,28.344818,-27.720559,64.440737,0.734911,2
381,549,0.770275,12.0,0.281721,28.346250,-27.720559,64.440737,0.734911,3
382,549,0.642990,5.0,0.281726,28.344620,-27.720559,64.440737,0.734911,4


#### save to correct folder based on group

In [16]:
os.getcwd()

'/Users/jackiebeltran/Documents/GitHub/RP_Learning/1_scripts/model_1'

In [17]:
### HEALTHY CONTROLS
final_df.to_csv('../../3_results/model_1/HC_fitting_results.csv') 

In [23]:
### MDD
final_df.to_csv('../../3_results/model_1/MDD_fitting_results.csv') 

In [72]:
### All subjects 
final_df.to_csv('../../3_results/model_1/N77_fitting_results.csv') 

#### keep parameters with greatest likelihood

In [73]:
df_sorted = final_df.sort_values(['sub_ID', 'likelihood_per_trial'], ascending=[True, False])
df_highest_likelihood = df_sorted.groupby('sub_ID').first().reset_index()
df_highest_likelihood

,sub_ID,initial_alpha_conditions,initial_beta_conditions,fit_alpha,fit_beta,log_likelihood,BIC_model1,likelihood_per_trial,Iteration
0,401,0.734983,15.0,0.222628,5.801811,-37.695205,84.390029,0.657812,5
1,402,0.899162,27.0,0.318157,3.120021,-46.106023,101.211665,0.599123,5
2,408,0.464348,13.0,0.000001,1.000000,-62.383247,133.766127,0.500000,4
3,409,0.239406,19.0,0.005874,50.000000,-44.362792,97.725203,0.610840,1
4,410,0.792170,33.0,0.427147,2.368726,-50.932154,120.851030,0.567842,4
...,...,...,...,...,...,...,...,...,...
72,545,0.174594,10.0,0.384965,2.861874,-48.946085,118.162669,0.580512,5
73,546,0.761961,17.0,1.000000,1.000000,-59.707369,131.709473,0.515089,2
74,547,0.191714,28.0,0.771373,50.000000,-27.034013,63.067645,0.740538,5
75,548,0.445622,42.0,0.304173,1.000000,-60.329166,129.657951,0.511543,5


In [19]:
### HEALTHY CONTROLS 
df_highest_likelihood.to_csv('../../3_results/model_1/HC_parameters.csv')

In [25]:
### MDD 
df_highest_likelihood.to_csv('../../3_results/model_1/MDD_parameters.csv')

In [74]:
### All subjects 
df_highest_likelihood.to_csv('../../3_results/model_1/N77_parameters.csv') 

### Extract RPEs

In [27]:
def likelihood_mle_outputs(alpha, actions, reward, condition):
        
    q_value = np.ones((3,2))*0.0 
    T = len(actions)
    #L = 0  
    RPE = []
    
    for t in range(T): # for every trial 
        
        # # compute choice probabilities of picking an action based on soft max 
        # p = np.exp(beta * q_value[condition[t]-1,:]) / (np.exp(beta * q_value[condition[t]-1,:])).sum()

        # # compute choice probability for actual choice based on the probability computed above by condition 
        # choiceProb = p[actions[t]-1]

        # # sum of the natural log of each individual choice probability to get the prob of a whole dataset (90 trials)
        # L += np.log(choiceProb) 

        # update values with q learning, index for t and update by condition 
        if ((condition[t] == 1) | (condition[t] == 2)):
            rpe = reward[t] - q_value[condition[t]-1, actions[t]-1]
            q_value[condition[t]-1, actions[t]-1] = q_value[condition[t]-1, actions[t]-1] + alpha * (rpe)
            RPE.append(rpe)
        elif condition[t] == 3:
            rpe_n = 0 
            RPE.append(rpe_n)
        
    return RPE   


In [28]:
def lat_vars(filename):
    
    a = os.path.basename(filename)
    subID = os.path.splitext(a)[0]

    # load data & subset for columns of interest
    sub_df = pd.read_csv(filename)
    
    # Obtain vars of interest (actions, rewards, conditions)  
    condition = sub_df["condition"]
    actions = sub_df['actions']
    reward = sub_df['rewards']
    
    fit_alpha = sub_df['fit_alpha'].iloc[0]
    #fit_beta = sub_df['fit_beta'].iloc[0]
        
    # extract RPEs
    rpes = likelihood_mle_outputs(fit_alpha, actions, reward, condition)
    
    results_df = pd.DataFrame(rpes, columns = ['RPE'])
    results_df['sub_ID'] = subID
    results_df = results_df[['sub_ID', 'RPE']]

    results_df.to_csv(f'../../3_results/model_1/RPEs/{subID}_RPE.csv')


##### load behavioral data

###### HEALTHY CONTROLS

In [40]:
df1_files = glob.glob('../../0_data/healthy/*.csv') 
df_highest_likelihood = pd.read_csv(f'/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/HC_parameters.csv')

###### MDD

In [54]:
df1_files = glob.glob('../../0_data/mdd/*.csv') 
df_highest_likelihood = pd.read_csv(f'/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/MDD_parameters.csv')

In [55]:
# empty list to store the loaded dataframes
df1_dataframes = []

# load and append each df1 dataframe to the list
for file in df1_files:
    
    df = pd.read_csv(file)
    
    col_name = ['sub_ID', 'actions','rewards', 'condition']   
    df = df[col_name]    
    
    df1_dataframes.append(df)

combined_df = pd.concat(df1_dataframes, ignore_index=True)

In [56]:
# load parameters with greatest likelihood

cols = ['sub_ID', 'fit_alpha', 'fit_beta']

df_highest_likelihood = df_highest_likelihood[cols]

In [57]:
# create one dataframe with behavioral data & parameters for each subject

params_df = pd.merge(combined_df, df_highest_likelihood)

In [58]:
# save dataframe with participant data & parameters into individual subject dataframes

grouped = params_df.groupby('sub_ID')

output_directory = '/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params'

for group_name, group_df in grouped:
    file_name = os.path.join(output_directory, f'{group_name}.csv')  # Generate a unique file name for each group
    group_df.to_csv(file_name, index=False)

##### extract RPEs

In [59]:
os.getcwd()

'/Users/jackiebeltran/Documents/GitHub/RP_Learning/1_scripts/model_1'

In [62]:
hlist = glob.glob('/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/5*.csv')
mlist = glob.glob('/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/4*.csv')

print(len(hlist))
print(len(mlist))

38
39


In [63]:
#for name in mlist:
for name in mlist:
    print(name)
    lat_vars(name)

/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/431.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/425.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/419.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/418.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/424.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/430.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/432.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/427.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/423.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/436.csv
/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_1/final_params/422.csv
/Users/jackiebeltran/